In [2]:
#import libraries

import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize


import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


In [3]:
# df of greek population/district for wikipedia

df = pd.read_html('https://el.wikipedia.org/wiki/%CE%9A%CE%B1%CF%84%CE%AC%CE%BB%CE%BF%CE%B3%CE%BF%CF%82_%CE%B4%CE%AE%CE%BC%CF%89%CE%BD_%CF%84%CE%B7%CF%82_%CE%95%CE%BB%CE%BB%CE%AC%CE%B4%CE%B1%CF%82')[0]

#Clean df
df.drop(["Έκταση (km²)",'Μετ.(±%)','Απ.(2001)','Μετ.(±%).1','Απ.(1991)',"Αποκεντρωμένη Διοίκηση"],axis=1,inplace=True)
df.drop(df[df.Νομός != "Αττικής"].index,inplace=True)
df.drop(df[df.Πυκνότητα == "***"].index,inplace=True)
df.drop(['Νομός'],axis=1,inplace=True)
df.reset_index(inplace=True)
df.drop(columns='index',inplace=True)
df.rename(columns={"Δήμος": "NAME","Πυκνότητα": "DENSITY","Απ.(2011)": "POPULATION"},inplace=True)
df = df.applymap(lambda s:s.upper() if type(s) == str else s)


for i,j in df.iterrows():
    if df.loc[i,"POPULATION"].is_integer():
        df.loc[i,"POPULATION"]=df.loc[i,"POPULATION"]/1000
df["POPULATION"]=df["POPULATION"]*1000


df.DENSITY=df.DENSITY.astype("float64")
for i,j in df.iterrows():
    if df.loc[i,"DENSITY"].is_integer():
        df.loc[i,"DENSITY"]=df.loc[i,"DENSITY"]/1000
df["DENSITY"]=df["DENSITY"]*1000


# Sort df
df.sort_values(by='DENSITY', inplace=True, ascending=False)
df=df.head()

# CHANGE GREEK TONE
old=["Ά",'Έ','Ή',"Ί","Ό","Ύ","Ώ"]
new=["Α",'Ε','Η',"Ι","Ο","Υ","Ω"]
df = df.replace(old,new,regex=True)

In [4]:
df

,NAME,DENSITY,POPULATION
41,ΚΑΛΛΙΘΕΑΣ,21187.6,100641.0
42,ΝΕΑΣ ΣΜΥΡΝΗΣ,20760.2,73076.0
34,ΝΕΑΣ ΙΩΝΙΑΣ,19181.1,67134.0
22,ΑΘΗΝΑΙΩΝ,17026.8,664046.0
10,ΠΕΙΡΑΙΑ,15058.7,163688.0


In [5]:
df = df.set_index('NAME')
df = df.rename(index={'ΠΕΙΡΑΙΑ':'ΠΕΙΡΑΙΩΣ'})
df.reset_index(inplace=True)
df

,NAME,DENSITY,POPULATION
0,ΚΑΛΛΙΘΕΑΣ,21187.6,100641.0
1,ΝΕΑΣ ΣΜΥΡΝΗΣ,20760.2,73076.0
2,ΝΕΑΣ ΙΩΝΙΑΣ,19181.1,67134.0
3,ΑΘΗΝΑΙΩΝ,17026.8,664046.0
4,ΠΕΙΡΑΙΩΣ,15058.7,163688.0


In [11]:
# get and clear GeoJSON

ath_geo = json.load(open('ATH.txt', encoding='utf-8'))

import itertools

features = []

for name in df.NAME:
    for k,v in itertools.groupby([x for x in ath_geo['features'] if x['properties']['GREEKNAME'] == name]):  
        features.append(k)
    
ath_geo['features'] = features



In [19]:
threshold_scale = np.linspace(df['DENSITY'].min(),
                              df['DENSITY'].max(),
                              6, dtype=float)
threshold_scale = threshold_scale.tolist() 
threshold_scale[-1] = threshold_scale[-1] + 1

map = folium.Map(location=[37.955894, 23.702099], zoom_start=11)


map.choropleth(
    geo_data=ath_geo,
    data=df,
    columns=['NAME','DENSITY'],
    key_on='feature.properties.GREEKNAME',
    threshold_scale=threshold_scale,
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Population Density',
    reset=True
)

map